In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)
training_file = 'aesop.txt'


In [3]:
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    return content

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word,_ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [4]:
training_data = read_data(training_file)
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [ ]:
print(dictionary)

In [5]:
# Parameters
learnin_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3
# Number of neurons in RNN cell
n_hidden = 512

In [6]:
# TF Graph
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_input, 1)
    # Create two layer LSTM
    rnn_cell = rnn.BasicLSTMCell(n_hidden)
    
    # Generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [7]:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learnin_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



In [16]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0
offset = random.randint(0, n_input+1)
end_offset = n_input + 1
while step < training_iters:
    if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)
    symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
    symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
    
    symbols_out_onehot = np.zeros([vocab_size], dtype=float)
    symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
    symbols_out_onehot = np.reshape(symbols_out_onehot, [1, -1])
    _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
    step += 1
    offset += (n_input+1)
    if step % display_step == 0:
        symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
        symbols_out = training_data[offset + n_input]
        symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval(session=sess))]
        print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))

['nobody', 'spoke', '.'] - [then] vs [.]
['at', 'one', 'another'] - [and] vs [looked]
['old', 'mouse', 'got'] - [up] vs [the]
['general', 'applause', ','] - [until] vs [well]
['should', 'always', 'know'] - [when] vs [we]
['a', 'ribbon', 'round'] - [the] vs [by]
['therefore', ',', 'to'] - [propose] vs [,]
['approach', ',', 'we'] - [could] vs [her]
['the', 'sly', 'and'] - [treacherous] vs [consists]
['which', 'he', 'thought'] - [would] vs [what]
[',', 'and', 'some'] - [said] vs [this]
['they', 'could', 'take'] - [to] vs [common]


IndexError: index 204 is out of bounds for axis 0 with size 204